## Train decision tree

In [84]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

# plt.figure(figsize=[10,10])


df_data = pd.read_csv(R".\in\disease-symptom_preprocessed2.csv", index_col=0)
# print(df_data.loc['asthma'])
# merge shortness of breath and dyspnea 
df_data['shortness of breath'] = (df_data['dyspnea'] + df_data['shortness of breath']).clip(lower=0, upper=1)
df_data = df_data.drop(columns='dyspnea')
df_data = df_data.drop(columns='night sweat')
df_data = df_data.drop(columns='cystic lesion')
df_data = df_data.drop(columns='night sweat')
df_data = df_data.drop(index='TOTAL')
# print(df_data.loc['failure heart congestive'])

X = df_data.to_numpy()

# y = df_data.index.to_list()
y = list(range(len(df_data.index.to_list())))  # 0 - 400
# print(y)
clf = DecisionTreeClassifier(criterion='entropy', random_state=0).fit(X, y)

feature_names = df_data.columns.to_list()

class_names = df_data.index.to_list()
# print(class_names[7])
# print(list(zip(feature_names,X[7])))


In [85]:
clf.predict([X[7]])

array([7])

## Visualize tree
Todo: make graphical visualization

from sklearn.tree import plot_tree
plt.figure(figsize=[10,10])
plot_tree(clf, fontsize=10)
fig = plt.gcf()
fig.savefig('tree.png')


remove extraneous info using method from here https://stackoverflow.com/questions/44821349/python-graphviz-remove-legend-on-nodes-of-decisiontreeclassifier

In [86]:
from sklearn.tree import export_graphviz
import pydot
import re

d = None  # max depth

export_graphviz(clf, 
                out_file=f'out/tree{d}.dot', 
                impurity=False, 
                feature_names=feature_names,
                class_names=class_names,
                max_depth=d)

PATH = f'out/tree{d}.dot'
f = pydot.graph_from_dot_file(PATH)[0].to_string()
# print(len(f), '\n', f[:1000])
# todo: modify this regex 
f = re.sub(r'(\\nvalue = \[.*?\])', '', f)  # get rid of nvalue = [anychar, lazy]
f = f.replace(' <= 0.5', '?')  # change decision to a question
f = f.replace('headlabel="True"', 'headlabel="No"')  # change to yes no rather than <=0.5 true false
f = f.replace('headlabel="False"', 'headlabel="Yes"')
f = f.replace(R'samples = 1\nclass = ', R'***\n')  # change text of leaf node
# print("============================")
# print(len(f), '\n', f[:1000])

with open(f'out/tree_modified{d}.dot', 'w') as file:
    file.write(f)

In [87]:
import graphviz
from IPython.display import Image
savepath = graphviz.render('dot', 'png', f'out/tree_modified{d}.dot')

Image(url=savepath)
# This gives a pretty good representation of the tree

from sklearn.tree.export import export_text

r = export_text(decision_tree, feature_names=df_data.columns.to_list())
print(r)